<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_two_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Collaborative Task Solving with Multiple Agents and Human Users

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate an application involving multiple agents and human users to work together and accomplish a task. `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. We create multiple `UserProxyAgent` instances that can represent different human users.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
# %pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for an environment variable of a specified name ("OAI_CONFIG_LIST" in this example), which needs to be a valid json string. If that variable is not found, it looks for a json file with the same name. It filters the configs by models (you can filter by other keys as well).

The json looks like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-4",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-08-01-preview"
    },
    {
        "model": "gpt-4-32k",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-08-01-preview"
    }
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

## Construct Agents

We define `ask_expert` function to start a conversation between two agents and return a summary of the result. We construct an assistant agent named "assistant_for_expert" and a user proxy agent named "expert". We specify `human_input_mode` as "ALWAYS" in the user proxy agent, which will always ask for feedback from the expert user.

In [6]:
def ask_expert(message):
    developer = autogen.AssistantAgent(
        name="developer",
        llm_config={
            "temperature": 0,
            "config_list": config_list,
        },
    )
    tester = autogen.UserProxyAgent(
        name="tester",
        human_input_mode="ALWAYS",
        code_execution_config={"work_dir": "expert"},
    )

    tester.initiate_chat(developer, message=message)
    tester.stop_reply_at_receive(developer)
    # expert.human_input_mode, expert.max_consecutive_auto_reply = "NEVER", 0
    # final message sent from the expert
    tester.send("summarize the solution and explain the answer in an easy-to-understand way", developer)
    # return the last message the expert received
    return tester.last_message()["content"]


We construct another assistant agent named "assistant_for_student" and a user proxy agent named "student". We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent. We set the `functions` in `AssistantAgent` and `function_map` in `UserProxyAgent` to use the created `ask_expert` function.

For simplicity, the `ask_expert` function is defined to run locally. For real applications, the function should run remotely to interact with an expert user.

In [4]:
assistant_for_student = autogen.AssistantAgent(
    name="assistant_for_student",
    system_message="You are a helpful assistant. Reply TERMINATE when the task is done.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "temperature": 0,
        "functions": [
            {
                "name": "ask_expert",
                "description": "ask expert when you can't solve the problem satisfactorily.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask expert. Ensure the question includes enough context, such as the code and the execution result. The expert does not know the conversation between you and the user unless you share the conversation with the expert.",
                        },
                    },
                    "required": ["message"],
                },
            }
        ],
    }
)

student = autogen.UserProxyAgent(
    name="student",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "student"},
    function_map={"ask_expert": ask_expert},
)

[autogen.oai.client: 11-29 15:04:52] {74} WARNING - openai client was provided with an empty config_list, which may not be intended.


## Perform a task

We invoke the `initiate_chat()` method of the student proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. The conversation will finish if you don't provide any feedback (by pressing Enter directly). Before the "TERMINATE" signal, the student proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [5]:
# the assistant receives a message from the student, which contains the task description
student.initiate_chat(
    assistant_for_student,
    message="""Find $a + b + c$, given that $x+y \\neq -1$ and 
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
""",
)

student (to assistant_for_student):

Find $a + b + c$, given that $x+y \neq -1$ and 
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant_for_student (to student):

To find $a + b + c$, we can add the three given equations together. 

Adding the left-hand sides of the equations gives us $ax + by + c + a + bx + cy + ay + b + cx = 2ax + 2bx + 2cx + ay + by + c$.

Adding the right-hand sides of the equations gives us $x + 7 + 2x + 6y + 4x + y = 7x + 7 + 7y$.

Setting these two equal to each other, we get $2ax + 2bx + 2cx + ay + by + c = 7x + 7 + 7y$.

We can simplify this to $2a + 2b + 2c + y(a + b) = 7x + 7 + 7y$.

Since $x + y \neq -1$, we can divide both sides by $x + y$ to get $2a + 2b + 2c = 7 + 7(x + y)$.

Solving for $a + b + c$, we get $a + b + c = \frac{7 + 7(x + y)}{2} = 3.5 + 3.5(x + y)$.

So, $a + b + c = 3.5 + 3.5(x + y)$.

----------

expert (to assistant_for_expert):

The user asked to find $a + b + c$, given that $x+y \neq -1$ and the equations 

\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.

I tried to solve this by adding the three equations together and simplifying, but I'm not sure if my approach is correct. Could you please check my solution and provide the correct one if mine is wrong?

--------------------------------------------------------------------------------
assistant_for_expert (to expert):

Sure, I can help with that. However, I need to know your solution first to check it. Could you please provide it? If you haven't solved it yet, I can solve it for you. 

Here is the plan:
1. I will add the three equations together.
2. I will simplify the equation and solve for $a + b + c$.

Let's start with the first step. We add the three equations together:

\begin{align}
(ax + by + c) + (a + bx + cy) + (ay + b + cx) & = (x + 7) + (2x + 6y) + (4x + y) \

expert (to assistant_for_expert):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    from sympy import symbols, Eq, solve
ModuleNotFoundError: No module named 'sympy'


--------------------------------------------------------------------------------
assistant_for_expert (to expert):

I apologize for the confusion. It seems that the 'sympy' module is not installed in your Python environment. You can install it using pip, which is a package manager for Python. Here is the command to install sympy:

```sh
pip install sympy
```

After installing sympy, you can run the previous Python code again.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is sh)...


KeyboardInterrupt: 

When the assistant needs to consult the expert, it suggests a function call to `ask_expert`. When this happens, a line like the following will be displayed:

***** Suggested function Call: ask_expert *****
